In [73]:
log_file = 'logs/mri_ct_s1_l0/02_DenseCL_Medical/ft_test_5k_mix_abdominal_mri_densecl_5_fold.log'
csv_file = log_file.replace('.log', '.csv')
with open(log_file, 'r') as f:
    log_data = f.readlines()
    for i in range(len(log_data)):
        log_data[i] = log_data[i].replace('\n', '')

In [74]:
import re
start_indice = []
end_indice = []
for i in range(len(log_data)):
    # start_flag = re.match(r'====== EVAL EVAL_FOLD \d MODEL_EPOCH \d{5,6} START ======', log_data[i]) # for test
    start_flag = re.match(r'====== EVAL EVAL_FOLD \d MODEL_EPOCH \d{4} START ======', log_data[i]) # for ft_test
    if start_flag is not None:
        start_indice.append(i)
    # end_flag = re.match(r'====== EVAL EVAL_FOLD \d MODEL_EPOCH \d{5,6} END ======', log_data[i])
    end_flag = re.match(r'====== EVAL EVAL_FOLD \d MODEL_EPOCH \d{4} END ======', log_data[i])
    if end_flag is not None:
        end_indice.append(i)
assert len(start_indice) == len(end_indice)

In [75]:
eval_fold_list = []
model_epoch_list = []
right_kindey_dice_list = []
left_kindey_dice_list = []
mean_dice_list = []
for s,e in zip(start_indice, end_indice):
    assert int(log_data[s].split(' ')[3])==int(log_data[e].split(' ')[3])
    eval_fold = int(log_data[s].split(' ')[3])
    eval_fold_list.append(eval_fold)
    assert int(log_data[s].split(' ')[5])==int(log_data[e].split(' ')[5])
    model_epoch = int(log_data[s].split(' ')[5])
    model_epoch_list.append(model_epoch)
    class_match_status = False
    mean_match_status = False
    for i in log_data[s:e]:
        class_dice_flag = re.match(r'mar_val batches classDice: \[([0-9.]+)([ ]+)([0-9.]+)([ ]*)\]', i)
        if class_dice_flag is not None and not class_match_status:
            right_kindey_dice = float(class_dice_flag.group(1))
            left_kindey_dice = float(class_dice_flag.group(3))
            right_kindey_dice_list.append(right_kindey_dice)
            left_kindey_dice_list.append(left_kindey_dice)
            class_match_status = True
        elif class_dice_flag is not None and class_match_status:
            raise ValueError(f'classDice has been matched more than once from index {s} to index {e}.')
        mean_dice_flag = re.match(r'mar_val batches meanDice:([ ]+)([0-9.]+)([ ]*)', i)
        if mean_dice_flag is not None and not mean_match_status:
            mean_dice = float(mean_dice_flag.group(2))
            mean_dice_list.append(mean_dice)
            mean_match_status = True
        elif mean_dice_flag is not None and mean_match_status:
            raise ValueError(f'meanDice has been matched more than once from index {s} to index {e}.')
    if not class_match_status:
        right_kindey_dice_list.append(None)
        left_kindey_dice_list.append(None)
    if not mean_match_status:
        mean_dice_list.append(None)
print(len(eval_fold_list), len(model_epoch_list), len(right_kindey_dice_list), len(left_kindey_dice_list), len(mean_dice_list))

25 25 25 25 25


In [76]:
import pandas as pd
data_dict = {
    'eval fold': eval_fold_list,
    'model epoch': model_epoch_list,
    'right kindey dice': right_kindey_dice_list,
    'left kindey dice': left_kindey_dice_list,
    'mean dice': mean_dice_list
    }
df = pd.DataFrame.from_dict(data_dict)
df.to_csv(csv_file, index=False)